In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder

train_df =pd.read_csv('data/modified_train.csv')
test_df = pd.read_csv('data/modified_test.csv')

test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

# train_df.head()

In [16]:
categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    te = TargetEncoder(cols=[i])
    train_x[i] = te.fit_transform(train_x[i], train_y)
    test_x[i] = te.transform(test_x[i])
    

['요일', '기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']

In [17]:
train_x = train_x.drop(columns='Unnamed: 0')
test_x = test_x.drop(columns='Unnamed: 0')

display(train_x)
display(test_x)

,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,4.627926,4.712888,4.712878,3.817650,2019,1,1,0,4.726704,4.541610,4.282449,4.671841,4.599599
1,4.627926,4.779150,4.712878,3.817650,2019,1,1,0,4.726704,4.618441,4.738938,4.671841,4.599599
2,4.627926,4.712888,4.712878,3.817650,2019,1,1,1,4.726704,4.727300,4.842715,4.671841,4.599599
3,4.627926,4.712888,4.712878,4.944597,2019,1,1,2,4.726704,4.687669,4.208920,4.671841,4.599599
4,4.627926,4.712888,4.712878,4.944597,2019,1,1,4,4.726704,4.889534,4.549091,4.671841,4.599599
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,4.600906,4.712888,4.712878,4.944597,2021,12,31,19,4.726704,4.727300,4.913420,4.882281,5.006142
39605,4.600906,4.712888,4.712878,4.944597,2021,12,31,19,4.726704,4.618441,4.293963,4.671841,4.599599
39606,4.600906,4.712888,4.712878,4.944597,2021,12,31,21,4.726704,4.618441,4.648867,4.882281,5.006142
39607,4.600906,4.712888,4.712878,4.944597,2021,12,31,22,4.726704,4.618441,4.672002,4.065962,4.599599


,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,4.920811,4.712888,4.712878,3.817650,2022,1,1,1,4.726704,4.727300,4.881657,4.882281,5.006142
1,4.920811,4.712888,4.712878,3.817650,2022,1,1,1,4.726704,4.727300,4.563008,4.671841,4.599599
2,4.920811,4.712888,4.712878,4.944597,2022,1,1,4,4.726704,4.727300,4.945578,4.882281,5.006142
3,4.920811,4.712888,4.712878,4.944597,2022,1,1,4,4.726704,4.727300,4.438172,4.671841,4.599599
4,4.920811,4.712888,4.712878,4.944597,2022,1,1,6,4.726704,4.618441,4.738938,4.882281,5.006142
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.920811,4.712888,4.712878,4.944597,2022,12,31,18,4.726704,4.623681,4.492281,4.671841,6.555670
10959,4.920811,4.712888,4.712878,4.944597,2022,12,31,18,4.726704,4.727300,4.928144,4.671841,4.599599
10960,4.920811,4.712888,4.712878,4.944597,2022,12,31,20,4.726704,4.727300,5.355036,4.671841,4.599599
10961,4.920811,4.712888,4.712878,4.944597,2022,12,31,20,4.726704,4.727300,5.079498,4.882281,4.863035


In [8]:
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool

train_preds = np.zeros(len(train_x))
test_preds = np.zeros(len(test_x))

skf = KFold(n_splits=5, shuffle=True, random_state=42)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_index, valid_index) in enumerate(skf.split(train_x, train_y)):
    dtrain = Pool(data=train_x.values[train_index], label=train_y.values[train_index])
    dvalid = Pool(data=train_x.values[valid_index], label=train_y.values[valid_index])
    bst = CatBoostRegressor(
        iterations=1000,
        objective="RMSE",
        learning_rate=0.01,
        l2_leaf_reg=6,
        depth=4,
        random_seed=42,
        subsample=0.7,
        bagging_temperature=0.23,
        od_type="Iter",
    )
    bst.fit(X=dtrain, eval_set=dvalid)
    test_preds += bst.predict(Pool(test_x.values)) / skf.n_splits


sample_submission = pd.read_csv("data/sample_submission.csv")
baseline_submission = sample_submission.copy()
baseline_submission["ECLO"] = test_preds.astype(int)
baseline_submission.to_csv("result/cb_te.csv", index=False)

0:	learn: 3.2193328	test: 3.1535298	best: 3.1535298 (0)	total: 8.58ms	remaining: 8.57s
1:	learn: 3.2184098	test: 3.1526821	best: 3.1526821 (1)	total: 16.8ms	remaining: 8.39s
2:	learn: 3.2174846	test: 3.1518359	best: 3.1518359 (2)	total: 22.9ms	remaining: 7.62s
3:	learn: 3.2165640	test: 3.1509430	best: 3.1509430 (3)	total: 28.6ms	remaining: 7.11s
4:	learn: 3.2157487	test: 3.1501740	best: 3.1501740 (4)	total: 33.7ms	remaining: 6.71s
5:	learn: 3.2149633	test: 3.1493972	best: 3.1493972 (5)	total: 38.3ms	remaining: 6.34s
6:	learn: 3.2141151	test: 3.1485764	best: 3.1485764 (6)	total: 42.3ms	remaining: 6s
7:	learn: 3.2132227	test: 3.1477484	best: 3.1477484 (7)	total: 46ms	remaining: 5.7s
8:	learn: 3.2123608	test: 3.1468597	best: 3.1468597 (8)	total: 50.3ms	remaining: 5.53s
9:	learn: 3.2115960	test: 3.1461301	best: 3.1461301 (9)	total: 54.1ms	remaining: 5.35s
10:	learn: 3.2108178	test: 3.1453819	best: 3.1453819 (10)	total: 58ms	remaining: 5.21s
11:	learn: 3.2100732	test: 3.1446523	best: 3.1446

In [20]:
import lightgbm as lgb
# lightgbm custom metric 적용하기

def RMSLE_LGBM(y, t):
    t = t.get_label()
    # print(y.shape, t.shape)
    log_y = np.log1p(y)
    log_t = np.log1p(t)
    loss = np.sqrt(np.mean((log_y - log_t)**2))
    return 'rmsle', loss, False


params = {
    # 'objective': 'root_mean_squared_error',
    "boosting_type": "gbdt",
    "objective": "regression",
    "learning_rate": 0.1,
    "seed": 42,
    "lambda_l1": 5,
    "lambda_l2": 5,
    "max_depth": 3,
    "num_leaves": 20,
    "force_col_wise": True,
    "nthread": 24,
    "bagging_fraction": 0.6,
    "feature_fraction": 0.5,
}

train_preds = np.zeros(len(train_x))
test_preds = np.zeros(len(test_x))


# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_index, valid_index) in enumerate(skf.split(train_x, train_y)):
    # print(train_x.values[train_index].shape, train_x.values[train_index].shape, '@@@@@@')
    # print(train_x.values[valid_index].shape, train_y.values[valid_index].shape, '######')
    
    dtrain = lgb.Dataset(train_x.values[train_index], label=train_y.values[train_index], free_raw_data=False)
    dvalid = lgb.Dataset(train_x.values[valid_index], label=train_y.values[valid_index], free_raw_data=False)

    bst = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dvalid], feval=RMSLE_LGBM)
    # train_preds[valid_index] = bst.predict(dvalid)
    # test_preds += bst.predict(lgb.Dataset(test_x)) / skf.n_splits
    test_preds += bst.predict(test_x.values) / skf.n_splits

# submit
sample_submission = pd.read_csv("data/sample_submission.csv")
baseline_submission = sample_submission.copy()
baseline_submission["ECLO"] = test_preds.astype(int)
baseline_submission.to_csv("result/baseline_submit.csv", index=False)

baseline_submission['ECLO'].value_counts()


[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 31687, number of used features: 12
[LightGBM] [Info] Start training from score 4.732382
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

ECLO
4     6223
5     2200
3     2160
6      243
2       70
7       38
8       13
9        8
10       5
11       2
12       1
Name: count, dtype: int64

In [ ]:
# catboost bestTest = 3.153726339